In [6]:
import os
import sys
import dotenv
import yaml
from pathlib import Path
from IPython.display import Markdown as md

sys.path.append(os.path.expanduser("~/source/appland/SWE-bench"))

from appmap.navie.editor import Editor
from appmap.navie.format_instructions import xml_format_instructions
from appmap.navie.fences import extract_fenced_content

# Expand home path to full path
project_dir = Path(os.path.expanduser("~")) / "source" / "kgilpin" / "nova-flask"
os.chdir(project_dir)

dotenv.load_dotenv(".env.notebook", override=True)
plan_dir = project_dir / "notebooks" / "0002_garrison"

In [8]:
garrison_sql = """
create table Garrison (
    garrisonId                     int not null primary key auto_increment,
	gameId                         bigint unsigned /* Game */ not null,
	starId                         bigint unsigned /* Star */ not null,
	numShips                       int not null
);
"""
work_dir = plan_dir / "garrison_model"
navie = Editor(work_dir)
plan = navie.plan(garrison_sql, prompt="Generate an alembic model for the Garrison table")
# print(navie.files())
code = navie.generate()
md(code)


  Output is available at /Users/kgilpin/source/kgilpin/nova-flask/notebooks/0002_garrison/garrison_model/plan/plan.md
  Output is available at /Users/kgilpin/source/kgilpin/nova-flask/notebooks/0002_garrison/garrison_model/generate/generate.md


To create the Alembic model for the `Garrison` table and integrate it into the existing SQLAlchemy models, follow these steps:

1. **Define the Garrison Model:**
   - Add a new class `Garrison` in the `nova/models.py` file.
   - Define the columns `garrisonId`, `gameId`, `starId`, and `numShips` with appropriate data types and constraints.
   - Establish foreign key relationships with the `Game` and `Star` models.

2. **Update Alembic Migration Scripts:**
   - Generate a new Alembic migration script to create the `Garrison` table in the database.
   - Ensure the migration script includes the table definition and foreign key constraints.

### Step 1: Define the Garrison Model

Add the following code to `nova/models.py`:

<!-- file: /Users/kgilpin/source/kgilpin/nova-flask/nova/models.py -->
```python
class Garrison(db.Model):
    garrisonId = db.Column(db.Integer, primary_key=True, autoincrement=True)
    gameId = db.Column(db.Integer, db.ForeignKey("game.gameId"), nullable=False)
    starId = db.Column(db.Integer, db.ForeignKey("star.starId"), nullable=False)
    numShips = db.Column(db.Integer, nullable=False)

    def __repr__(self):
        return f"<Garrison {self.garrisonId}>"
```

### Step 2: Update Alembic Migration Scripts

Generate a new Alembic migration script using the following command:

```sh
alembic revision --autogenerate -m "Create Garrison table"
```

This command will create a new migration script in the `migrations/versions` directory. Open the generated script and ensure it includes the definition for the `Garrison` table. It should look something like this:

<!-- file: /Users/kgilpin/source/kgilpin/nova-flask/migrations/versions/<revision_id>_create_garrison_table.py -->
```python
from alembic import op
import sqlalchemy as sa

# revision identifiers, used by Alembic.
revision = '<revision_id>'
down_revision = '<previous_revision_id>'
branch_labels = None
depends_on = None

def upgrade():
    # ### commands auto generated by Alembic - please adjust! ###
    op.create_table('garrison',
    sa.Column('garrisonId', sa.Integer(), autoincrement=True, nullable=False),
    sa.Column('gameId', sa.Integer(), nullable=False),
    sa.Column('starId', sa.Integer(), nullable=False),
    sa.Column('numShips', sa.Integer(), nullable=False),
    sa.ForeignKeyConstraint(['gameId'], ['game.gameId'], ),
    sa.ForeignKeyConstraint(['starId'], ['star.starId'], ),
    sa.PrimaryKeyConstraint('garrisonId')
    )
    # ### end Alembic commands ###

def downgrade():
    # ### commands auto generated by Alembic - please adjust! ###
    op.drop_table('garrison')
    # ### end Alembic commands ###
```

### Step 3: Apply the Migration

Apply the migration to update the database schema:

```sh
alembic upgrade head
```

By following these steps, you will create an Alembic model for the `Garrison` table and integrate it into the existing SQLAlchemy models. This will ensure that the `Garrison` table is properly defined and managed within the project's database schema.